In [7]:
import os
import re
import json
import shutil
import textract
import datetime
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
import subprocess

In [8]:
""" 
Some of the code in this script is based on the following sources:
https://colab.research.google.com/drive/1RU0TysCxnoT1sjy3l__DGVdsPWLAG1wT#scrollTo=Yq09i0jnczdi
created by Alessandro Corvi
"""
def driversetup():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36")
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("lang=en")
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument('ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)
    return driver
def setup_directories(base_dir='pdfs'):
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    return base_dir

def getpage(url, pdf_dir, pdf_name):
    pdf_path = os.path.join(pdf_dir, pdf_name)

    if os.path.exists(pdf_path):
        print(f'\tFile skipped, already present. Filename: {pdf_name}')
        return pdf_path

    driver = driversetup()
    driver.get(url)

    try:
        filename = max([f for f in os.listdir('.') if f.endswith('.pdf')], key=os.path.getctime)
        shutil.move(filename, pdf_path)
    except Exception as e:
        print(f'File failed, refer to the following URL: {url}\nError: {str(e)}')
        driver.quit()
        return None

    driver.quit()
    return pdf_path

def main():
    output_dir = 'RootDirectoryOfPatentData'
    pdfs_dir = setup_directories(output_dir)

    claims = pd.read_excel('data/tree.xlsx')
    res = []
    for k in range(len(claims['EFSA Opinion Reference'])):
        i = claims['EFSA Opinion Reference'][k]
        claim = claims['Claim'][k]
        if pd.isna(i):
            continue
        for j in re.split(',', str(i)):
            args = re.split(';|:', j)
            if len(args) == 3:
                res.append({
                    'url': 'https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.' + args[0].strip() + '.' + args[2].strip(),
                    'claim': claim,
                    'index': k,
                    'args': args
                })

    for entry in tqdm(res):
        url = entry['url']
        claim = entry['claim']
        idx = entry['index']
        pdf_code = entry['args'][0].strip()+"_"+entry['args'][2].strip()
        
        # Directory for this specific PDF document
        pdf_dir = os.path.join(pdfs_dir, pdf_code)
        check = os.path.join(pdfs_dir,'Claim_Doc')
        if not os.path.exists(check):
            if not os.path.exists(pdf_dir):
                os.makedirs(pdf_dir)
        else:
            print(f"Folder for {pdf_code} already exists. Skipping PDF processing.")

        pdf_name = f'{pdf_code}.pdf'
        pdf_path = os.path.join(pdf_dir, pdf_name)
        
        if not os.path.exists(pdf_path):
            pdf_path = getpage(url, pdf_dir, pdf_name)

In [9]:
main()

  1%|          | 3/291 [00:15<22:44,  4.74s/it]

File failed, refer to the following URL: https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2011.2049
Error: max() arg is an empty sequence


 12%|█▏        | 35/291 [01:01<06:52,  1.61s/it]

File failed, refer to the following URL: https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2010.1764
Error: max() arg is an empty sequence


 15%|█▌        | 44/291 [01:16<07:51,  1.91s/it]

...

 92%|█████████▏| 267/291 [06:52<00:25,  1.08s/it]

File failed, refer to the following URL: https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2011.2074
Error: max() arg is an empty sequence


 93%|█████████▎| 272/291 [07:00<00:24,  1.30s/it]

File failed, refer to the following URL: https://efsa.onlinelibrary.wiley.com/doi/pdf/10.2903/j.efsa.2009.1229
Error: max() arg is an empty sequence


100%|██████████| 291/291 [07:07<00:00,  1.47s/it]
